In [17]:
import cv2                 #Τσιτώνη Ελένη 03447
import numpy as np
import mediapipe as mp
from tkinter import filedialog, messagebox
import tkinter as tk
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
import os
from fpdf import FPDF

class PhotoAlbum:  # Δημιουργία κλάσης που περιέχει λειτουργίες της εφαρμογής
    def __init__(self, root):  # Αρχικοποίηση
        self.root = root  # Δημιουργία του κύριου παραθύρου της εφαρμογής
        self.root.title("Photo Album")  # Ορισμός τίτλου του παραθύρου

        self.original_image = None  
        self.edited_image = None
        self.image_path = None
        self.album_folder = "album_images"
        if not os.path.exists(self.album_folder):
            os.makedirs(self.album_folder)

        self.canvas = tk.Canvas(self.root, width=600, height=400)  # Δημιουργία και τοποθέτηση καμβά
        self.canvas.grid(row=0, column=0, columnspan=5)

        self.load_button = tk.Button(root, text="Load Image", command=self.load_image)  # Δημιουργία και τοποθέτηση κουμπιού
        self.load_button.grid(row=1, column=0)

        self.save_button = tk.Button(root, text="Save Image", command=self.save_image)  # Δημιουργία και τοποθέτηση κουμπιού
        self.save_button.grid(row=1, column=1)

        self.reset_button = tk.Button(root, text="Reset Canvas", command=self.reset_canvas)  # Δημιουργία και τοποθέτηση κουμπιού
        self.reset_button.grid(row=1, column=2)

        self.filter_var = tk.StringVar(self.root)  # Δημιουργία και τοποθέτηση μενού επιλογής φίλτρων
        self.filter_var.set("Select Filter")
        self.filter_menu = tk.OptionMenu(self.root, self.filter_var, "COLOR", "BRIGHTNESS", "SMOOTH", "EDGE_ENHANCE")
        self.filter_menu.grid(row=1, column=3)

        self.apply_filter_button = tk.Button(root, text="Apply Filter", command=self.apply_filter)  # Δημιουργία και τοποθέτηση κουμπιού
        self.apply_filter_button.grid(row=1, column=4)

        self.add_hat_button = tk.Button(root, text="Add Hat", command=self.add_hat)  # Δημιουργία και τοποθέτηση κουμπιού
        self.add_hat_button.grid(row=2, column=0)

        self.add_glasses_button = tk.Button(root, text="Add Glasses", command=self.add_glasses)  # Δημιουργία και τοποθέτηση κουμπιού
        self.add_glasses_button.grid(row=2, column=1)

        self.detect_lines_button = tk.Button(root, text="Detect Lines", command=self.detect_lines)  # Δημιουργία και τοποθέτηση κουμπιού
        self.detect_lines_button.grid(row=3, column=0)

        self.detect_circles_button = tk.Button(root, text="Detect Circles", command=self.detect_circles)  # Δημιουργία και τοποθέτηση κουμπιού
        self.detect_circles_button.grid(row=3, column=1)
        
        self.create_album_button = tk.Button(root, text="Create Album", command=self.create_album)
        self.create_album_button.grid(row=3, column=2)

    def load_image(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.png;*.jpeg;*.jpg;*.tiff")])  # Υποστηριζόμενες μορφές αρχείων
        if not file_path:
            return

        try:  # Φόρτωση και εμφάνιση εικόνας
            self.original_image = Image.open(file_path)
            self.update_canvas(self.original_image)
        except Exception as e:
            messagebox.showerror("Error", f"Could not open image file: {e}")

    def save_image(self):
        if not self.edited_image:  # Έλεγχος για φορτωμένη εικόνα
            return
        file_path = filedialog.asksaveasfilename(defaultextension=".jpeg",
                                                 filetypes=[("JPEG files", "*.jpeg"),
                                                            ("PNG files", "*.png"),
                                                            ("TIFF files", "*.tiff")])  # Παράθυρο αποθήκευσης αρχείου
        if not file_path:
            return
        self.edited_image.save(file_path)  # Αποθήκευση εικόνας
        self.save_to_album(file_path)
        messagebox.showinfo("Save Image", "Image saved successfully!")

    def save_to_album(self, file_path):
        base_name = os.path.basename(file_path)
        album_path = os.path.join(self.album_folder, base_name)
        self.edited_image.save(album_path)

    def reset_canvas(self):  # Επαναφορά του καμβά και των μεταβλητών στην αρχική τους κατάσταση
        self.canvas.delete("all")
        self.original_image = None
        self.edited_image = None

    def update_canvas(self, image):
        self.canvas.delete("all")
        self.display_image = ImageTk.PhotoImage(image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.display_image)
        self.root.update_idletasks()

    def apply_filter(self):
        if not self.original_image:
            messagebox.showerror("Error", "No image loaded to apply filter.")
            return

        filter_type = self.filter_var.get()  # Ανακτά τον τύπο του φίλτρου που έχει επιλέξει ο χρήστης από το μενού επιλογής φίλτρων
        if filter_type == "Select Filter":
            messagebox.showerror("Error", "Please select a valid filter.")
            return

        filtered_image = self.original_image.copy()  # Αντιγραφή της αρχικής εικόνας πριν την εφαρμογή του φίλτρου
        if filter_type == "COLOR":
            enhancer = ImageEnhance.Color(filtered_image)
            filtered_image = enhancer.enhance(1.5)  # Αλλαγή έντασης χρώματος
        elif filter_type == "BRIGHTNESS":
            enhancer = ImageEnhance.Brightness(filtered_image)
            filtered_image = enhancer.enhance(1.5)  # Αλλαγή φωτεινότητας
        elif filter_type == "SMOOTH":
            filtered_image = filtered_image.filter(ImageFilter.SMOOTH)  # Εφαρμογή φίλτρου smoothing
        elif filter_type == "EDGE_ENHANCE":
            filtered_image = filtered_image.filter(ImageFilter.EDGE_ENHANCE)  # Εφαρμογή φίλτρου edge enhancement

        self.update_canvas(filtered_image)
        self.edited_image = filtered_image  # Ενημέρωση της επεξεργασμένης εικόνας

    def create_album(self):
        pdf = FPDF()
        for image_name in os.listdir(self.album_folder):
            # Βεβαιωθείτε ότι το αρχείο είναι εικόνα
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
                image_path = os.path.join(self.album_folder, image_name)
                pdf.add_page()
                pdf.image(image_path, x=10, y=10, w=190)
        album_path = os.path.join(self.album_folder, "album.pdf")
        pdf.output(album_path)
        messagebox.showinfo("Album Created", f"Album saved as {album_path}")

    def add_hat(self):
        if not self.original_image:
            messagebox.showerror("Error", "No image loaded to add hat.")
            return
        try:
            self.load_hat()
            filtered_image = detect_and_add_hat(self.original_image.copy(), self.hat)
            self.update_canvas(filtered_image)
            self.edited_image = filtered_image
        except Exception as e:
            messagebox.showerror("Error", f"Could not add hat: {e}")

    def add_glasses(self):
        if not self.original_image:
            messagebox.showerror("Error", "No image loaded to add glasses.")
            return
        try:
            self.load_glasses()
            filtered_image = detect_and_add_glasses(self.original_image.copy(), self.glasses)
            self.update_canvas(filtered_image)
            self.edited_image = filtered_image
        except Exception as e:
            messagebox.showerror("Error", f"Could not add glasses: {e}")

    def load_hat(self):
        self.hat = ensure_alpha_channel(load_overlay_image('hat.png'))

    def load_glasses(self):
        self.glasses = ensure_alpha_channel(load_overlay_image('glasses.png'))

    def detect_lines(self):
        if not self.original_image:
            messagebox.showerror("Error", "No image loaded to detect lines.")
            return
        try:
            open_cv_image = cv2.cvtColor(np.array(self.original_image), cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150, apertureSize=3)
            lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
            if lines is not None:
                for rho, theta in lines[:, 0]:
                    a = np.cos(theta)
                    b = np.sin(theta)
                    x0 = a * rho
                    y0 = b * rho
                    x1 = int(x0 + 1000 * (-b))
                    y1 = int(y0 + 1000 * (a))
                    x2 = int(x0 - 1000 * (-b))
                    y2 = int(y0 - 1000 * (a))
                    cv2.line(open_cv_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            self.edited_image = Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))
            self.update_canvas(self.edited_image)
        except Exception as e:
            messagebox.showerror("Error", f"Could not detect lines: {e}")

    def detect_circles(self):
        if not self.original_image:
            messagebox.showerror("Error", "No image loaded to detect circles.")
            return
        try:
            open_cv_image = cv2.cvtColor(np.array(self.original_image), cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, minDist=50,
                                       param1=50, param2=30, minRadius=1, maxRadius=30)
            if circles is not None:
                circles = np.uint16(np.around(circles))
                for i in circles[0, :]:
                    cv2.circle(open_cv_image, (i[0], i[1]), i[2], (0, 255, 0), 2)
                    cv2.circle(open_cv_image, (i[0], i[1]), 2, (0, 0, 255), 3)
            self.edited_image = Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))
            self.update_canvas(self.edited_image)
        except Exception as e:
            messagebox.showerror("Error", f"Could not detect circles: {e}")

def load_overlay_image(file_path):
    image = cv2.imread(file_path, -1)
    if image is None:
        file_path = filedialog.askopenfilename(title=f"Select {file_path} file",
                                               filetypes=[("Image Files", "*.png;*.jpeg;*.jpg;*.tiff")])
        image = cv2.imread(file_path, -1)
        if image is None:
            raise FileNotFoundError(f"Could not load {file_path}. Make sure the file exists and is in the correct format.")
    return image

def ensure_alpha_channel(image):
    if image.shape[2] == 3:
        alpha_channel = np.ones((image.shape[0], image.shape[1], 1), dtype=image.dtype) * 255
        image = np.concatenate((image, alpha_channel), axis=2)
    return image

def add_overlay(img, overlay, x, y, w, h):
    overlay = cv2.resize(overlay, (w, h))
    for i in range(h):
        for j in range(w):
            if overlay[i, j][3] != 0:
                img[y + i, x + j] = overlay[i, j][:3]

def detect_and_add_hat(image, hat):
    open_cv_image = np.array(image.convert('RGB'))
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    mp_face_detection = mp.solutions.face_detection
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = open_cv_image.shape
                (startX, startY, endX, endY) = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                                                int((bboxC.xmin + bboxC.width) * iw), int((bboxC.ymin + bboxC.height) * ih))
                hat_height = int(0.5 * (endY - startY))
                add_overlay(open_cv_image, hat, startX, startY - hat_height, endX - startX, hat_height)
    return Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))

def detect_and_add_glasses(image, glasses):
    open_cv_image = np.array(image.convert('RGB'))
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    mp_face_detection = mp.solutions.face_detection
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = open_cv_image.shape
                (startX, startY, endX, endY) = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                                                int((bboxC.xmin + bboxC.width) * iw), int((bboxC.ymin + bboxC.height) * ih))
                glasses_height = int(0.15 * (endY - startY))
                eye_y = startY + int(0.3 * (endY - startY))
                add_overlay(open_cv_image, glasses, startX, eye_y, endX - startX, glasses_height)
    return Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))

def detect_and_draw_lines(image):
    open_cv_image = np.array(image.convert('RGB'))
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(open_cv_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))

def detect_and_draw_circles(image):
    open_cv_image = np.array(image.convert('RGB'))
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, minDist=50,
                               param1=50, param2=30, minRadius=1, maxRadius=30)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv2.circle(open_cv_image, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(open_cv_image, (i[0], i[1]), 2, (0, 0, 255), 3)
    return Image.fromarray(cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB))

def run_app():
    root = tk.Tk()
    app = PhotoAlbum(root)
    root.mainloop()

if __name__ == "__main__":
    run_app()
